# CSC3831: Coursework Part I

# Anomaly Detection

## Setup

In [ ]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
houses = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/IMPUTATION/TARGET-DATASETS/ORIGINAL/houses.csv', header=0)

In [ ]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   median_house_value  20640 non-null  float64
 1   median_income       20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   latitude            20640 non-null  float64
 8   longitude           20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


## Plotting utility

In [ ]:
## plotting utility

from math import ceil

##
## type= {boxplot, kdeplot}
##
def plot_distributions(data, columns, type='boxplot', title=None):

    #print("plotting columns {c}".format(c=list(columns)))
    
    if type not in {'boxplot', 'kdeplot', 'histplot'}:
        print("type= {boxplot, kdeplot, histplot} only are supported")
        return

    ## grid size depends on number of columns
    ## max 4 columns in the grid
    maxCols  = 4
    
    if len(columns) < 4:
        numCols = len(columns)
    else:
        numCols = maxCols
    numRows = ceil(len(columns) / 4)
    
    #print("grid is {0}x{1}".format(numRows, numCols))

    fig, axs = plt.subplots(numRows, numCols)
    fig.suptitle(title)
    fig.set_figwidth(5*numCols)
    fig.set_figheight(3*numCols)
    fig.tight_layout(pad=5.0)

    #print(axs)

#         handle special axes
    if numRows == 1 and numCols == 1:
        c = columns[0]
        # axes is a scalar
        if type == 'boxplot':
            sns.boxplot(data=data, x=c, ax=axs)
        elif type == 'kdeplot':
            sns.kdeplot(data=data, x=c, ax=axs)
        else:
            sns.histplot(data=data, x=c, ax=axs)
        axs.set_title(c)

    elif numRows == 1:
        i = 0
        # axes is a 1D array
        for c in columns:
#         print("column {c}".format(c=c))
            if type == 'boxplot':
                sns.boxplot(data=data, x=c, ax=axs[i])
            elif type == 'kdeplot':
                sns.kdeplot(data=data, x=c, ax=axs[i])
            else:
                sns.histplot(data=data, x=c, ax=axs[i])
            axs[i].set_title(c)
            i = i+1
        
    else:
    # general case of a 2D grid    
        i=j=0    
        for c in columns:
            #print("column {c}".format(c=c))
            if type == 'boxplot':
                #print("plotting on axes [{0},{1}]".format(i,j))
                sns.boxplot(data=data, x=c, ax=axs[i,j])
            elif type == 'kdeplot':
                sns.kdeplot(data=data, x=c, ax=axs[i,j])
            else:
                sns.histplot(data=data, x=c, ax=axs[i,j])

            axs[i,j].set_title(c)
            j = j+1
            if j == 4:
                i = i+1
                j= 0


## Descriptive analytics: let's start by looking at raw statistics for the features in this dataset. What sort of story are they telling?

#### Building a detailed and informative map of Californian household distribution
The most interesting thing about our dataset, in my opinion, is the fact that instead of having, say, the area name as the records' unique identifier, we have the longitude and latitude of the area. In terms of data visualisation this is quite significant, as it allows us to essentially build a geographic map of the distribution of households in California with ease.

In [ ]:
plt.figure(figsize=(9,9))

plt.scatter(houses["longitude"], houses["latitude"], zorder=10)

plt.grid()
plt.title("Geographic Distribution of Households", size=20)

By lowering the opacity of the data points we can also clearly see the locations that are more densly populated versus the ones that are not.

In [ ]:
plt.figure(figsize=(9,9))

plt.scatter(houses["longitude"], houses["latitude"], alpha=0.4, zorder=10)

plt.grid()
plt.title("Geographic Distribution of Households", size=20)

Furthermore, we can colour code household areas by their median value

In [ ]:
plt.figure(figsize=(9,9))

plt.scatter(houses["longitude"], houses["latitude"], alpha=0.4, c=houses["median_house_value"], cmap=plt.get_cmap("inferno"), zorder=10)

plt.colorbar(label="Median House Value")
plt.grid()
plt.title("Geographic Distribution of Households",size=20)

Comparing this plot to a map of california, you could see that the two clusters of areas where the vast majority of expensive households are, are in or close to San Francisco (cluster of bright dots in the North) and Los Angeles (cluster of bright dots in the South). Another notable location is Sacramento (cluster of slightly bright dots Northeast of San Francisco). 

Considering these are some of the biggest cities in California, it is safe to conclude that location plays a major role in determining house value.

Now, along with the median house value we visualise other features to see if we can notice any more correlations.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 16))

# Multipliers are needed so the data points do not appear too big or too small.
# What we care about is how big a circle is compared to another in the same plot,
# so this does not impact any possible observations.
multipliers = [[0.04, 15], [2, 0.015]]
columns = [["population", "median_income"], ["housing_median_age", "total_rooms"]]
column_names = [["Population", "Median Income"], ["Housing Median Age", "Total Rooms"]]
   
for i in range(2):
    for j in range(2):

        mappable = ax[i, j].scatter(houses["longitude"], houses["latitude"], alpha=0.4, 
                                    c=houses["median_house_value"], s=houses[columns[i][j]] * multipliers[i][j], 
                                    cmap=plt.get_cmap("inferno"), zorder=10, label=column_names[i][j])

        fig.colorbar(mappable=mappable, ax=ax[i, j], label="Median House Value")

        ax[i, j].grid()
        ax[i, j].legend()

fig.suptitle("Geographic Distribution of Households", size=20)

The above plots show the following:

* Location is likely the most closely related feature to house value.

* Population density is also important.

* House age does not seem to be closely related to house value. The median house age both in the locations with the highest median house values and outside those is fairly mixed.

* Initially it looks like a connection between the total rooms of the houses in an area with house value does exist, but even if it does it is a meaningless one. Across all the different areas there are different numbers of households, so the total number of rooms (and by extension the number of total bedrooms) cannot be compared, i.e. it does not indicate bigger houses but more often than not just a different number houses in the area. We will come back to this later.

#### Median house value compared with population per area, median income, median household age and total rooms

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=[10, 10])

ax[0, 0].scatter(houses["median_house_value"], houses["population"], alpha=0.4)
ax[0, 0].set_title("Value-Population")
ax[0, 1].scatter(houses["median_house_value"], houses["median_income"], alpha=0.4)
ax[0, 1].set_title("Value-Income")
ax[1, 0].scatter(houses["median_house_value"], houses["housing_median_age"], alpha=0.4)
ax[1, 0].set_title("Value-Age")
ax[1, 1].scatter(houses["median_house_value"], houses["total_rooms"], alpha=0.4)
ax[1, 1].set_title("Value-Rooms")

plt.show()

This plot confirms what we saw earlier in the geographic distribution. There is no direct correlation between any of these features except median income.

In [ ]:
houses.corr()["median_house_value"][1:].sort_values(ascending=False)

Pearson's correlation agrees.

Before moving onto feature normalisation, let's have a look at the box plot of each of the different features:

In [ ]:
plot_distributions(houses, houses.columns)

housing_median_age, latitude and longitude are evenly distributed, so when removing outliers we will not mess with these features.

## Feature Normalisation 

Some of the features need to be normalised before any conclusion can be drawn.

As mentioned previously, the total rooms feature is not very helpful. For the same reason, total bedrooms is not either. And while population density is relevant as we have noticed, the population itself is not. So, we create three new features: _rooms_per_household_, _bedrooms_per_household_ and _people_per_household_

In [ ]:
houses_normalised = houses.copy()

houses_normalised["rooms_per_household"] = (houses["total_rooms"] / houses["households"])
houses_normalised["bedrooms_per_household"] = (houses["total_bedrooms"] / houses["households"])
houses_normalised["people_per_household"] = (houses["population"] / houses["households"])

houses_normalised.head()

Let's see how the new features compare against median house value:

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=[10, 10])

ax[0, 0].scatter(houses_normalised["median_house_value"], houses_normalised["rooms_per_household"], alpha=0.4)
ax[0, 0].set_title("Value-Rooms per Household")
ax[0, 1].scatter(houses_normalised["median_house_value"], houses_normalised["bedrooms_per_household"], alpha=0.4)
ax[0, 1].set_title("Value-Bedrooms per Household")
ax[1, 0].scatter(houses_normalised["median_house_value"], houses_normalised["people_per_household"], alpha=0.4)
ax[1, 0].set_title("Value-People per Household")

plt.show()

At first glance, all the new features appear to have a negative correlation with median house value if we ignore some clear outliers (for the last one, if at all).

However, the vast majority of data points are at the bottom of each chart. What if we zoom in?

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=[10, 10])

ax[0, 0].scatter(houses_normalised["median_house_value"], houses_normalised["rooms_per_household"], alpha=0.3)
ax[0, 0].set_title("Value-Rooms per Household")
ax[0, 0].set_ylim(0, 20)
ax[0, 1].scatter(houses_normalised["median_house_value"], houses_normalised["bedrooms_per_household"], alpha=0.4)
ax[0, 1].set_title("Value-Bedrooms per Household")
ax[0, 1].set_ylim(0, 5)
ax[1, 0].scatter(houses_normalised["median_house_value"], houses_normalised["people_per_household"], alpha=0.4)
ax[1, 0].set_title("Value-People per Household")
ax[1, 0].set_ylim(0, 20)

plt.show()

In the cases of bedrooms and people per household, the negative correlation still holds, but in rooms per household not so much.

In [ ]:
houses_normalised.corr()["median_house_value"][1:].sort_values(ascending=False)

Pearson's correlation not only disagrees with a negative correlation between rooms_per_household and value, but even suggests a notable positive one (though not necessarily a strong one). This has to do with the outliers I mentioned before, as well as most data points probably being on the right side of the chart.

## Record Identification

Now that we have analysed the normalised features, it is time to identify outliers.

My thought process for choosing outliers is fairly simple. Look at feature comparisons with median house value and if there is a visible increase or decrease in data points as median house value increases, clear out data points that are not clustered up with most of the others and which make the correlation weaker than it should be.

#### Detect outliers in median house value - income plot

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["median_income"])

In [ ]:
outliers = houses_normalised[(houses_normalised["median_house_value"] > 500000) & (houses["median_income"] < 4)]
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] > 350000) & (houses_normalised["median_income"] < 2.5)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] > 250000) & (houses_normalised["median_income"] < 1.8)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] > 150000) & (houses_normalised["median_income"] < 1)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] < 75000) & (houses_normalised["median_income"] > 5)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] < 150000) & (houses_normalised["median_income"] > 8)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] < 250000) & (houses_normalised["median_income"] > 7)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[houses_normalised["median_income"] > 10]]).drop_duplicates()

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["median_income"], zorder=9)
plt.scatter(outliers["median_house_value"], outliers["median_income"], color="red", zorder=10)

#### Detect outliers in median house value - people per household plot

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["people_per_household"])

In [ ]:
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] > 330000) & (houses_normalised["people_per_household"] > 3.8)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] > 220000) & (houses_normalised["people_per_household"] > 4.5)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[houses_normalised["people_per_household"] > 7]]).drop_duplicates()

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["people_per_household"], zorder=9)
plt.scatter(outliers["median_house_value"], outliers["people_per_household"], color="red", zorder=10)
plt.ylim(0, 10)

#### Detect outliers in median house value - bedrooms per household plot

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["bedrooms_per_household"])

In [ ]:
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] > 350000) & (houses_normalised["bedrooms_per_household"] > 1.2)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["median_house_value"] > 150000) & (houses_normalised["bedrooms_per_household"] > 1.5)]]).drop_duplicates()
outliers = pd.concat([outliers, houses_normalised[(houses_normalised["bedrooms_per_household"] > 2.5) | (houses_normalised["bedrooms_per_household"] < 0.75)]]).drop_duplicates()

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["bedrooms_per_household"], zorder=9)
plt.scatter(outliers["median_house_value"], outliers["bedrooms_per_household"], color="red", zorder=10)
plt.ylim(0, 5)

#### Detect outliers in median house value - population

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["population"])

In [ ]:
outliers = pd.concat([outliers, houses_normalised[houses_normalised["population"] > 9000]]).drop_duplicates()

In [ ]:
plt.scatter(houses_normalised["median_house_value"], houses_normalised["population"], zorder=9)
plt.scatter(outliers["median_house_value"], outliers["population"], color="red", zorder=10)
plt.ylim(0, 10000)

#### Create clean dataframe

In [ ]:
houses_normalised_clean = pd.concat([outliers, houses_normalised]).drop_duplicates(keep=False)
print(f"Number of outliers: {len(outliers)}, records left: {len(houses_normalised_clean)}")

In [ ]:
houses_normalised_clean.corr()["median_house_value"][1:].sort_values(ascending=False)

With the outliers are removed, the correlation between median house value and median income is even stronger, the one with rooms_per_household has nearly doubled, bedrooms_per_household has almost quadrupled and the one with people_per_household is over ten times larger. In other words, all correlations that we observed previously have become stronger and more apparent.

#### The outliers

Printing the index number of all outliers.

In [ ]:
outliers.index.tolist()

## Applying LOF to see if the results align with the empirical analysis

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from numpy import where

#### Performing LOF

In [ ]:
# n_neighbours value was chosen to give the best results in median_house_value correlations
clf = LocalOutlierFactor(n_neighbors=12)
y_pred = clf.fit_predict(houses_normalised)
outlier_index = where(y_pred == -1)

In [ ]:
outliers_lof = houses_normalised.iloc[outlier_index]
houses_normalised_clean_lof = pd.concat([houses_normalised, outliers_lof]).drop_duplicates(keep=False)

print(f"Number of outliers: {len(outliers_lof)}, records left: {len(houses_normalised_clean_lof)}")

#### Looking at results

The correlations for median_house_value with the other features now look like this:

In [ ]:
houses_normalised_clean_lof.corr()["median_house_value"][1:].sort_values(ascending=False)

Difference with the manual outlier detection results:

In [ ]:
diff = houses_normalised_clean_lof.corr()["median_house_value"][1:] - houses_normalised_clean.corr()["median_house_value"][1:]
diff.sort_values(ascending=False)

The difference between the manual and LOF correlations is miniscule.

It is good that the correlations are similar, but how many outliers are actually the same?

In [ ]:
mutual = len(outliers.merge(outliers_lof))
print("Number of mutual outliers: {}, percentage of LOF outliers: {:3.4}%".format(mutual, mutual/ len(outliers_lof) * 100))

41.42% of LOF outliers are the also found in the manual detection outlier set. This is most likely due to the fact that I mostly picked outliers in terms of what would help median_house_value's correlations become more apparent, whereas LOF picked outliers within the individual features.

## Overall Conclusions

I manually extracted 1126, as opposed to 565 with LOF and got similar results in terms of correlations. Not to mention, only 234 of those were mutual. This probably means that I extracted a fair bit more than I needed too.

Let's break down each of median_house_value's correlations and their meaning:

* Median income: Both in manual outlier detection and in LOF we ended up with a stronger correlation between median_house_value and median_income than we started with in the houses dataset, and in all cases it is a strong correlation. This makes sense, richer people buy more expensive houses.

* Rooms per household: Though not as strong in LOF as it was in manual detection, rooms_per_household has a decent positive correlation with median_house_value. Again this makes sense, bigger house -> more expensive.

* Total rooms / Total bedrooms / Population: As I mentioned previously, any correlation with these features is irrelevant, as different areas have a different number of households, therefore these features were normalised.

* Housing median age: Neither the original dataset, the manually cleansed dataset or the LOF-cleansed dataset discovered a correlation between house value and age, and no such correlation was found in the geographic distribution visualisation either.

* Households: We used this feature to normalise other features in order to find correlations.

* People per household: A fairly strong negative correlation was found between house value and people per household both by manual detection and LOF. This could indicate that rich people in California tend to have smaller families.

* Bedrooms per household: My manual detection and LOF disagree here. I have found a decent negative correlation between house value and bedrooms per household, which would back up the correlation with people per household, LOF found no correlation.

* Longitude / Latitude: Location is an important determining factor of house value, as it became evident when visualising the geographic distribution of the areas. The areas near the coast had the highest median house values, but since it is not a clear east/west or north/south divide, we cannot see it in the numbers.

# Data Imputation and Machine Learning

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [ ]:
houses_missing = pd.read_csv("https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/IMPUTATION/TARGET-DATASETS/CORRUPTED/HOUSES/houses_0.5_MAR.csv", header=0)

In [ ]:
houses_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          20640 non-null  int64  
 1   median_house_value  20640 non-null  float64
 2   median_income       10320 non-null  float64
 3   housing_median_age  10320 non-null  float64
 4   total_rooms         20640 non-null  float64
 5   total_bedrooms      20640 non-null  float64
 6   population          10320 non-null  float64
 7   households          20640 non-null  float64
 8   latitude            20640 non-null  float64
 9   longitude           20640 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 1.6 MB


In [ ]:
houses_missing.isnull().sum()

Unnamed: 0                0
median_house_value        0
median_income         10320
housing_median_age    10320
total_rooms               0
total_bedrooms            0
population            10320
households                0
latitude                  0
longitude                 0
dtype: int64

We can see that the median_income, housing_median_age and population features are all missing 50% of their values, so we are going to have to impute them.

There is also a useless extra feature, so let's get rid of it.

In [ ]:
houses_missing.drop(columns = houses_missing.columns[0], axis=1, inplace=True)

## Imputation

We have to take care of the missing values, so we will use KNN imputation and MICE imputation and see which resulting dataset gives better results in training.

### KNN Imputation

In [ ]:
houses_missing_copy_knn = houses_missing.copy()

knn_imputer = KNNImputer()
houses_imputed_knn = knn_imputer.fit_transform(houses_missing_copy_knn)

houses_imputed_knn = pd.DataFrame(houses_imputed_knn, columns=[houses_missing_copy_knn.columns])

In [ ]:
houses_imputed_knn.isnull().sum()

median_house_value    0
median_income         0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
latitude              0
longitude             0
dtype: int64

In [ ]:
houses_imputed_knn.head()

,median_house_value,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
0,452600.0,8.3252,41.0,880.0,129.0,697.8,126.0,37.88,-122.23
1,358500.0,8.3014,21.0,7099.0,1106.0,2508.6,1138.0,37.86,-122.22
2,352100.0,7.2574,52.0,1467.0,190.0,656.2,177.0,37.85,-122.24
3,341300.0,5.6431,52.0,1274.0,235.0,539.0,219.0,37.85,-122.25
4,342200.0,3.8462,52.0,1627.0,280.0,565.0,259.0,37.85,-122.25


### MICE Imputation

In [ ]:
houses_missing_copy_mice = houses_missing.copy()

mice_imputer = IterativeImputer()
houses_imputed_mice = mice_imputer.fit_transform(houses_missing_copy_mice)

houses_imputed_mice = pd.DataFrame(houses_imputed_mice, columns=[houses_missing_copy_mice.columns])

In [ ]:
houses_imputed_mice.isnull().sum()

median_house_value    0
median_income         0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
latitude              0
longitude             0
dtype: int64

As expected, there are no missing values left, let's have a look at the resulting dataset:

In [ ]:
houses_imputed_mice.head()

,median_house_value,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
0,452600.0,8.3252,41.0,880.0,129.0,-31.062649,126.0,37.88,-122.23
1,358500.0,8.3014,21.0,7099.0,1106.0,3232.172162,1138.0,37.86,-122.22
2,352100.0,7.2574,52.0,1467.0,190.0,276.419700,177.0,37.85,-122.24
3,341300.0,5.6431,52.0,1274.0,235.0,354.710926,219.0,37.85,-122.25
4,342200.0,3.8462,52.0,1627.0,280.0,565.000000,259.0,37.85,-122.25


In [ ]:
print(houses_imputed_mice[houses_imputed_mice["median_income"] < 0].count().sum())
print(houses_imputed_mice[houses_imputed_mice["housing_median_age"] < 0].count().sum())
print(houses_imputed_mice[houses_imputed_mice["population"] < 0].count().sum())

57
52
102


MICE has given imputed negative values in quite a few places, however, as long as the mean of the features is within logical bounds it should not negatively affect the training.

### First look at imputation results

Now that imputation is competed, let's compare:

In [ ]:
print(float(houses["median_income"].mean()))
print(float(houses["housing_median_age"].mean()))
print(float(houses["population"].mean()))

3.8706710029069766
28.639486434108527
1425.4767441860465


In [ ]:
print(float(houses_imputed_knn["median_income"].mean()))
print(float(houses_imputed_knn["housing_median_age"].mean()))
print(float(houses_imputed_knn["population"].mean()))

3.9341242936046514
28.216531007751936
1322.8441569767442


In [ ]:
print(float(houses_imputed_mice["median_income"].mean()))
print(float(houses_imputed_mice["housing_median_age"].mean()))
print(float(houses_imputed_mice["population"].mean()))

3.92076779133454
26.376294748892526
1363.758561158823


The MICE imputed dataset's median_income feature's mean is slightly closer to the original dataset's, but the difference is minimal, which is good since median_income is one of the most important features.

The KNN imputed dataset is almsot spot on with the mean for housing_median_age, whereas the MICE imputed dataset is a little off. This is not an important feature though.

The MICE imputed dataset is a fair bit closer to the original set's population feature's mean.

## Scaling

We should scale our data before training. To ensure our final results are not affected by the scaling method used, we will train each dataset using three different scalers and then compare the results.

### Scaling the KNN-Imputed dataset

In [ ]:
scaler_knn_standard = StandardScaler()
scaler_knn_minmax = MinMaxScaler()
scaler_knn_robust = RobustScaler()

scalers_knn = [scaler_knn_standard, scaler_knn_minmax, scaler_knn_robust]
houses_imputed_knn_scaled = []

for i, scaler_knn in enumerate(scalers_knn):
    houses_imputed_knn_scaled.append(scaler_knn.fit_transform(houses_imputed_knn))
    houses_imputed_knn_scaled[i] = pd.DataFrame(houses_imputed_knn_scaled[i], columns=[houses_missing_copy_knn.columns])

### Scaling the MICE-Imputed dataset

In [ ]:
scaler_mice_standard = StandardScaler()
scaler_mice_minmax = MinMaxScaler()
scaler_mice_robust = RobustScaler()

scalers_mice = [scaler_mice_standard, scaler_mice_minmax, scaler_mice_robust]
houses_imputed_mice_scaled = []

for i, scaler_mice in enumerate(scalers_mice):
    houses_imputed_mice_scaled.append(scaler_mice.fit_transform(houses_imputed_mice))
    houses_imputed_mice_scaled[i] = pd.DataFrame(houses_imputed_mice_scaled[i], columns=[houses_missing_copy_mice.columns])

## Training

### Training the KNN-Imputed dataset

In [ ]:
X_knn = []
Y_knn = []

for dataset_knn in houses_imputed_knn_scaled:
    X_knn.append(dataset_knn.drop(columns=["median_house_value"]))
    Y_knn.append(dataset_knn["median_house_value"])

In [ ]:
mse_knn = []
r2_knn = []

for i in range(len(X_knn)):
    x_train_knn, x_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn[i], Y_knn[i], test_size=0.2, random_state=1)

    M1 = LinearRegression()
    M1.fit(x_train_knn, y_train_knn)
    y_pred_knn = M1.predict(x_test_knn)

    mse_knn.append(np.sqrt(mean_squared_error(y_test_knn, y_pred_knn)))
    r2_knn.append(r2_score(y_test_knn, y_pred_knn))

### Training the MICE-Imputed dataset

In [ ]:
X_mice = []
Y_mice = []

for dataset_mice in houses_imputed_mice_scaled:
    X_mice.append(dataset_mice.drop(columns=["median_house_value"]))
    Y_mice.append(dataset_mice["median_house_value"])

In [ ]:
mse_mice = []
r2_mice = []

for i in range(len(X_mice)):
    x_train_mice, x_test_mice, y_train_mice, y_test_mice = train_test_split(X_mice[i], Y_mice[i], test_size=0.2, random_state=1)

    M2 = LinearRegression()
    M2.fit(x_train_mice, y_train_mice)
    y_pred_mice = M2.predict(x_test_mice)

    mse_mice.append(np.sqrt(mean_squared_error(y_test_mice, y_pred_mice)))
    r2_mice.append(r2_score(y_test_mice, y_pred_mice))

## Measuring and comparing performance

In [ ]:
scalers = ["StandardScaler", "MinMaxScaler", "RobustScaler"]

### KNN Performance

In [ ]:
for i, scaler in enumerate(scalers):
    print("Results using " + scaler + ":")
    print("\tMSE: " + str(mse_knn[i]))
    print("\tR2 Score:" + str(r2_knn[i]) + "\n")

Results using StandardScaler:
	MSE: 0.5444858121327648
	R2 Score:0.6990466199965089

Results using MinMaxScaler:
	MSE: 0.12954534936356743
	R2 Score:0.6990466199965089

Results using RobustScaler:
	MSE: 0.4329354248546351
	R2 Score:0.699046619996509



### MICE Performance

In [ ]:
for i, scaler in enumerate(scalers):
    print("Results using " + scaler + ":")
    print("\tMSE: " + str(mse_mice[i]))
    print("\tR2 Score:" + str(r2_mice[i]) + "\n")

Results using StandardScaler:
	MSE: 0.4960101919822987
	R2 Score:0.750248968391306

Results using MinMaxScaler:
	MSE: 0.11801191541896265
	R2 Score:0.7502489683913058

Results using RobustScaler:
	MSE: 0.3943911455781411
	R2 Score:0.750248968391306



### Comparison

In all cases, MICE imputation outperformed KNN, so we know beyond all doubt that the scaling method did not affect our results.

KNN uses Euclidean distance and neighbours to work out how to impute missing values, but with features missing as much as 50% of values, it is not as efficient as MICE.

MICE imputation is also more suited for data MAR, which is the case here, as it iteratively trains on each of the missing values.

Conclusion: For a situation such as this MICE is the better imputation method to use.